We can set our preferences for our electives manually here, possible options are:

* **gift**: Yes, No. *Yes meaning that a student does a Global Immersion Field Trip hence the credit cap is reduced by 11.*

* **extra**: Yes, No. *Yes meaning that a student wants to take the maximum amount of electives based on their credit cap.* 

* **subject**: All, Strategy & Entrepreneurship, Marketing, Finance, Management Science & Operations, Organisational Behaviour, Accounting, Economics. 

* **experience**: Overall, Learning Experience, Coursework, Class Participation, Prior Knowledge. 

In [1]:
gifts = "Yes"
extra = "Yes"
subject = "Marketing"
experience = "Overall"

# ELECTIVE SELECTION OPTIMISATION
#### Decision Analytics and Modelling: Study Group 6 
## Introduction

LBS students are faced with an important decision during their master's programme: **choosing their electives**. From a pool of 60 courses, students have to identify where their interests lie, what they want their schedule to look like in the upcoming months, what type of evaluation they want to be scored on, whether to opt for more technical or managerial courses, what they value from courses at LBS... And all of this has to be decided by long and tedious reading and analysing webpages with information regarding each course. 

To make this process a little simpler, adaptable and less hour intensive, we have decided to build an **optimisation tool that includes student's preferences and inclinations in the elective selection process**. To build it, the tool considers:
- The total amount of extra credits allowed for each student (whether they take a GIFT or want to max out credits or not)
- The subject area of each course
- The term and type of scheduling for each elective
- The weight that students give to each of the 7 evaluation scores of electives captured in previous years

From those data points, the model will optimise to choose electives with the highest average evaluation scores given the weighst, and give a suggestion to students about which electives to choose. *But first, we created a comprehensive data set that allowed us to carry out the optimisation.*

In [2]:
import pandas as pd
import numpy as np

# lOAD DATASET
data = pd.read_excel("Clean DAM Dataset.xlsx")

## Data Exploration

In [3]:
data.columns

Index(['Name', 'Number', 'Stream', 'Credit', 'Subject', 'Faculty', 'Term',
       'Schedule', 'Weekday', 'Start Time', 'End Time', 'Start Time 2',
       'End Time 2', 'Mon 8:15', 'Mon 12:45', 'Mon 16:00', 'Mon 19:00',
       'Tue 8:15', 'Tue 12:45', 'Tue 16:00', 'Tue 19:00', 'Wed 8:15',
       'Wed 12:45', 'Wed 16:00', 'Wed 19:00', 'Thu 8:15', 'Thu 12:45',
       'Thu 16:00', 'Thu 19:00', 'Fri 8:15', 'Fri 12:45', 'Fri 16:00',
       'Fri 19:00', 'Sat 8:15', 'Sat 12:45', 'Sat 16:00', 'Sat 19:00',
       'Block Week Number', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7',
       'Comment'],
      dtype='object')

There are columns in our dataset.

In [4]:
len(data["Name"].unique())

60

There are a total of 60 unique electives.

In [5]:
data["Term"].unique()

array(['SUM', 'SPR', 'SUM '], dtype=object)

There are a total of 3 terms for our electives.

In [6]:
data["Subject"].unique()

array(['Strategy & Entrepreneurship', 'Marketing', 'Finance',
       'Management Science & Operations', 'Organisational Behaviour',
       'Accounting', 'Economics'], dtype=object)

There are 7 departments/subjects for our electives.

In [7]:
data["Schedule"].unique()

array(['5 Week', 'Block Week', 'Day', 'Modular ', 'Irregular Block Week',
       'Modular', 'Evening'], dtype=object)

There are a total of 7 types of schedules. There are only one entries for "Irregular Block Week" which is Entrepreneurship Summmer School electives.

In [8]:
data["Credit"].unique()

array([11,  6])

There are only two types of credits.

In [9]:
data.head()

,Name,Number,Stream,Credit,Subject,Faculty,Term,Schedule,Weekday,Start Time,...,Sat 19:00,Block Week Number,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Comment
0,Advanced Competitive Strategy,E492,A,11,Strategy & Entrepreneurship,Sendil Ethiraj,SUM,5 Week,"Tue, Thu",12:45:00,...,0,NaN,4.670000,4.83000,4.830000,4.500000,4.830000,4.670000,2.670000,NaN
1,Advanced Competitive Strategy,E492,B,11,Strategy & Entrepreneurship,Sendil Ethiraj,SUM,Block Week,NaN,NaN,...,0,10.0,3.560000,3.81000,3.440000,3.810000,3.800000,4.250000,3.000000,NaN
2,Advanced Marketing Strategy,E217,A,11,Marketing,Rob Waiser,SUM,Block Week,NaN,NaN,...,0,9.0,4.530000,4.58000,4.580000,4.360000,4.380000,4.500000,3.530000,NaN
3,AI in Marketing,E612,B,11,Marketing,Lil Mohan,SUM,5 Week,"Mon, Wed",16:00:00,...,0,NaN,4.175000,4.24500,4.186667,4.176667,4.180000,4.133333,2.768333,NaN
4,Alternative Credit Investing,E609,C,6,Finance,Alon Avner,SUM,5 Week,Mon,12:45:00,...,0,NaN,4.261441,4.32322,4.392458,4.394153,4.296186,4.410593,2.812119,By Department


From the 5 rows shown above, we can see that the dataset created has 46 columns with all information required for building the model. This data was gathered mostly from the Enrolment Management System webpage (https://ems.london.edu/CourseDirectory) and includes all 60 courses offered to Master's in Analytics and Management students in the 2021-2022 academic year. The columns describe:
- **Elective ID**: The columns 'Name', 'Number' and 'Stream' help identify the specific elective that is being referred to. 
- **Credits**: The 'Credit' column indicates whether they are a 6 or 11 credit course. 
- **Subject and Faculty**: They describe the area of study and teaching professor for each course. 
- **Schedule**: The columns 'Term'. 'Schedule', 'Weekday', 'Start Time', 'End Time', 'Block Week Number' and the following 24 columns are there to help us identify when each of the electives is happening in order to build a functional schedule. 
- **Evaluation scores**: 7 questions posed to students after finishing their elective modules evaluate the student satisfaction, and are saved on the data set for evaluation. The mean, maximum and minimum values are shown hereafter for all seven. 

In [10]:
data[["Q1","Q2","Q3","Q4","Q5","Q6","Q7"]].describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7
count,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000
mean,4.264561,4.330109,4.388946,4.390055,4.305673,4.411855,2.823919
std,0.362304,0.386743,0.437559,0.384477,0.390825,0.396707,0.281375
min,3.260000,2.870000,2.680000,3.090000,2.880000,2.670000,1.840000
25%,4.115000,4.172269,4.210000,4.230000,4.160000,4.250000,2.710000
50%,4.290000,4.380000,4.520000,4.408077,4.310000,4.449744,2.812119
75%,4.530000,4.580000,4.660000,4.670000,4.590000,4.710000,3.000000
max,4.890000,5.000000,5.000000,5.000000,5.000000,5.000000,3.530000


This table shows the distribution of evaluation score values, seperately by questions.

## Data Transformation

In [11]:
data.index = data["Name"] + " " + data["Stream"].fillna("")

## Model Building

Having created a comprehensive data set, we know go onto model building, which consists of creating our decision variables, our constraints (both must-have constraints and preference-related constraints) and model optimisation. 

In [12]:
import gurobipy as gp
from gurobipy import GRB,quicksum

In [13]:
m = gp.Model("electives")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


### Decision Variables

Our decision variables are whether to take an elective or not, so it would be a binary variable for each course:

$$ x_{course}$$

where x is binary.

In [14]:
# We create a list with all the course names
course = data.index.tolist()

# Then we create our binary decision variable, which will give a 1 if the elective is chosen and 0 if not.  
electives = m.addVars(course, vtype = GRB.BINARY, name = "electives")

### Must-Have Constraints

#### Stream Constraint

This constraint describes the fact two different streams of the same elective cannot be selected, i.e. "Advanced Competitive Strategy A" and "Advanced Competitive Strategy B" cannot be both taken. To create this elective, we differentiate between elective 'name' and elective 'stream'. 

In [15]:
from collections import Counter

multi_stream_course = list(dict(filter(lambda elem: elem[1] > 1, 
                                       dict(Counter(data["Name"].tolist())).items())).keys())
print(len(multi_stream_course))

12


The courses above have multiple streams for the same elective name. We can then set constraints on those by locating their index and setting the sum of these indices to less than or equal to 1, i.e. only one of them can be selected. In mathematical notation: 

$$ \sum_{stream}x_{stream} \leq 1 $$

where $x_{stream}$ are decision variables of the electives that are the same electives but are just of different streams.

*This constraint is used throughout our constraints creation, changing the type of electives evaluated within the summation by using different for loops.* 

In this case, the for loop helps us evaluate the sum of elective streams with the same name obtained from the `multi_stream_course` list created above. 

In [16]:
for name in multi_stream_course:
    index = data[data["Name"] == name].index
    m.addConstr(quicksum(electives[s] for s in index) <= 1)

### Timetable Constraint

Following, we look for clashes in classes and set constraints so that our selected electives do not have a clashing schedule. 

Block weeks are hosted in weeks that are different from the rest so we can treat block weeks seperately from the rest electives. 
For other electives of other schedules, we create a sparse matrix that has binary values, indicating at what time these electives are held. 
Lastly, we also set a constraint on the terms, we cannot take more than 3 electives that are in the same term to prevent too much overload.

**Block Week**

We cannot have a clash in block weeks, i.e. take two electives that are on the same block week.
Using a for loop we evaluate all elective with a block week type teaching method. 

The constraint, in mathematical notation: 

$$ \sum_{block}x_{block} \leq 1 $$

where $x_{block}$ are decision variables of the electives that have same block week number.

In [17]:
# We create a list with all electives with block week numbers
block_week_clashes = list(dict(filter(lambda elem: elem[1] > 1, 
                                      dict(Counter(data["Block Week Number"].tolist())).items())).keys())
print(len(block_week_clashes))

for week_number in block_week_clashes:
    index = data[data["Block Week Number"] == week_number].index
    m.addConstr(quicksum(electives[b] for b in index) <= 1)

5


**Scheduling**

We cannot have a clash in scheduled time of classes. We will make use of our sparse matrix here. It is important to note that we can have classes with the same schedule in the case that they are in different terms (spring or summer). The mathematical expression for the created constraint: 

$$ \sum_{n \in allCourses}x_n*s_{tc}\leq 1 $$

where $s_{tc}$ is a vector in the one-hot matrix representing the schedules of our classes, column names are all possible schedule times (separated by time of day, weekday and term).

In [18]:
class_schedules = ['Mon 8:15', 'Mon 12:45', 'Mon 16:00', 'Mon 19:00','Tue 8:15', 'Tue 12:45', 'Tue 16:00', 
                   'Tue 19:00', 'Wed 8:15', 'Wed 12:45', 'Wed 16:00', 'Wed 19:00', 'Thu 8:15', 'Thu 12:45', 
                   'Thu 16:00', 'Thu 19:00', 'Fri 8:15', 'Fri 12:45', 'Fri 16:00', 'Fri 19:00', 'Sat 8:15', 
                   'Sat 12:45', 'Sat 16:00', 'Sat 19:00']
print(len(class_schedules))
print(len(data["Term"].unique().tolist()))

for term in data["Term"].unique().tolist():
    index = data[data["Term"] == term].index
    m.addConstrs(quicksum(electives[i]*data.loc[i,j] for i in index) <= 1 for j in class_schedules)

24
3


**Term**

We cannot have more than 3 electives in a single term. The mathematial expression for the created constraint:

$$ \sum_{term}x_{term} \leq 3 $$

where $x_{term}$ are decision variables of the electives that are in the same term

In [19]:
for term in data["Term"].unique().tolist():
    index = data[data["Term"] == term].index
    m.addConstr(quicksum(electives[i] for i in index) <= 3)

### Credit Constraint

Here, we are setting the credit constraint. As a basis, we are setting it as not taking GIFTs nor the extra credit, therefore, we will be limited to 44 total credits.

Furthermore, as we are not taking the extra credit, we will only be able to take 11 credit courses.

The mathematial expression for the created constraint:

$$ \sum_{n \in allCourses}x_{n} * credit_{n} \leq 44 $$

where $credit_n$ is the credit of elective $n$.

We also need to set a constraint for not being able to select a 6 credit elective unless we choose to take the extra 6 credit. This constraint can be changed based on user preference, it will be no in default, and in mathematical terms:

$$ \sum_{extra}x_{extra} = 0 $$

where $x_{extra}$ are decision variables of the 6 credit electives.

In [20]:
credit = 44

# Constraint on total credit
gift_credit = m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) <= credit)

# We cannot select 6 credit courses unless we choose to take the extra credit
index = data[data["Credit"] == 6].index
extra_credit = m.addConstr(quicksum(electives[i] for i in index) == 0)

## Preference Constraints

The following part allows for addition constraints based on user preferences. There will be 4 choosable constraints:

* Whether to take GIFTs

* Whether to 6 Extra Credit electives

* Subject Preferencs

* Experience Preferences (Adding weights on particular questions)

### GIFTs Constraint

We can choose whether to take the GIFTs by change the variable "gifts" to "Yes" or "No".

In [21]:
# Set if we are taking GIFTs
#gifts = "Yes"

### Extra Credit Constraint

Likewise, we can choose whether to take the extra 6 credit electives by change the variable "extra" to "Yes" or "No".

In [22]:
# Set if we are taking Extra Credit
#extra = "No"

### Add Constraints based on Credit Preferences

This will change our credit constraint and whether we can select 6 credit course constraint based on selected preferences. Base on our preference in whether to take GIFTs or the extra 6 credit elective, we will be setting different constraints:

If we are taking GIFTs and the extra 6 credit elective, we will be setting the following constraints:

$$ \sum_{n \in allCourses}x_{n} * credit_{n} = 39 $$

$$ \sum_{extra}x_{extra} = 1 $$

where $credit_n$ is the credit of elective $n$ and $x_{extra}$ are decision variables of the 6 credit electives.

If we are taking GIFTs and not taking the extra 6 credit elective, we will be setting the following constraints:

$$ \sum_{n \in allCourses}x_{n} * credit_{n} = 33 $$

$$ \sum_{extra}x_{extra} = 0 $$

where $credit_n$ is the credit of elective $n$ and $x_{extra}$ are decision variables of the 6 credit electives.

If we are not taking GIFTs but taking the extra 6 credit elective, we will be setting the following constraints:

$$ \sum_{n \in allCourses}x_{n} * credit_{n} = 50 $$

$$ \sum_{extra}x_{extra} = 1 $$

where $credit_n$ is the credit of elective $n$ and $x_{extra}$ are decision variables of the 6 credit electives.

If we are not taking GIFTs and not taking the extra 6 credit elective, we will be setting the following constraints:

$$ \sum_{n \in allCourses}x_{n} * credit_{n} = 44 $$

$$ \sum_{extra}x_{extra} = 0 $$

where $credit_n$ is the credit of elective $n$ and $x_{extra}$ are decision variables of the 6 credit electives.


In [23]:
# Add constraints based on preferences
m.remove(gift_credit)
m.remove(extra_credit)
index = data[data["Credit"] == 6].index

if gifts == "Yes" and extra == "Yes":
    # Credit Constraints
    credit = 39
    m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
    # 6 Credit Course Constraint
    m.addConstr(quicksum(electives[i] for i in index) == 1)
    
elif gifts == "Yes" and extra == "No":
    # Credit Constraints
    credit = 33
    m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
    # 6 Credit Course Constraint
    m.addConstr(quicksum(electives[i] for i in index) == 0)
    
elif gifts == "No" and extra == "Yes":
    # Credit Constraints
    credit = 50
    m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
    # 6 Credit Course Constraint
    m.addConstr(quicksum(electives[i] for i in index) == 1)
    
elif gifts == "No" and extra == "No":
    # Credit Constraints
    credit = 44
    m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
    # 6 Credit Course Constraint
    m.addConstr(quicksum(electives[i] for i in index) == 0)

### Subject Preference Constraint

Here, we can specific our subject preference to get more personalized elective selections. For our selected subject preference, we will assign at least half of the credits to that selected subject. In mathematical notations:

$$ \sum_{subject}x_{subject} * credit_{n} \geq \frac{TotalCredit}{2} $$

where $x_{subject}$ are the decision variables of electives in our preferred subject.

In [24]:
#subject = "Marketing"

if subject != "All":
    index = data[data["Subject"] == subject].index
    m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in index) >= credit/2)

## Objective

### Experience Preference

We are also able to select our preferred experience from our selected electives. This will affect our objective function that we are maximising. The experiences can be quantified by the evaluation questions asked:

1. Overall, how much do you think you have learnt from the course?

2. How well do you believe the course met its stated objectives?

3. How would you rate the overall effectiveness of the faculty?

4. How would you rate the timeliness of the feedback on course work/assignments from the instructors?

5. To what extent did the faculty provide useful feedback on course work/assignments?

6. How well did the faculty manage high quality standards for class participation?

7. How much previous knowledge of the subject did you have?

For example, if we enjoy class participation in class and we want electives that allows for that, we can only optimize based on the evaluation score of Q6 in our objective function.

We group our questions into 4 different experiences that we can choose from:

* Learning Experience: Q1, Q2, Q3

* Coursework: Q4, Q5

* Class Participation: Q6

* Prior Knowledge: Q7



In [25]:
#experience = "Coursework"

if experience == "Overall":
    weight_dict = {"Q1":1, "Q2":1, "Q3":1, "Q4":1, "Q5":1, "Q6":1, "Q7":1}
elif experience == "Learning Experience":
    weight_dict = {"Q1":1, "Q2":1, "Q3":1, "Q4":0, "Q5":0, "Q6":0, "Q7":0}
elif experience == "Coursework":
    weight_dict = {"Q1":0, "Q2":0, "Q3":0, "Q4":1, "Q5":1, "Q6":0, "Q7":0}
elif experience == "Class Participation":
    weight_dict = {"Q1":0, "Q2":0, "Q3":0, "Q4":0, "Q5":0, "Q6":1, "Q7":0}
elif experience == "Prior Knowledge":
    weight_dict = {"Q1":0, "Q2":0, "Q3":0, "Q4":0, "Q5":0, "Q6":0, "Q7":1}

With the question weight dictionary that are specify by our experience preference, we will then define our objective function based on the evaluation score. We will be maximising the following equation:

$$ \sum_{n \in courses}  x_{n} \left[ \left(\sum_{Q1 - Q6} weightDictionary * evaluationScore \right) - weightDictionary_{Q7} * evaluationScore_{Q7} \right] $$

where $weightDicitionary$ is obtained based on our experience preference.

We choose to minus evaluation score for Q7 because it is better that an elective requires less previous knowledge.

In [26]:
questions = ["Q1","Q2","Q3","Q4","Q5","Q6"]

m.setObjective(quicksum(data.loc[j,i]*electives[j]*weight_dict[i] for i in questions for j in course)-
               quicksum(data.loc[j,"Q7"]*electives[j]*weight_dict["Q7"] for j in course),
               GRB.MAXIMIZE)

# Optimize and Results

With the constraints, we can then optimize to get the best portfolio of electives based on our objection function.

In [27]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        if experience == "Overall":
            if credit == 33:
                print('\nAverage Overall Evaluation Score: %g' % (m.objVal/(5*3)))
            elif credit == 39 or credit == 44:
                print('\nAverage Overall Evaluation Score: %g' % (m.objVal/(5*4)))
            elif credit == 50:
                print('\nAverage Overall Evaluation Score: %g' % (m.objVal/(5*5)))
        elif experience == "Learning Experience":
            if credit == 33:
                print('\nAverage Learning Experience Evaluation Score: %g' % (m.objVal/(3*3)))
            elif credit == 39 or credit == 44:
                print('\nAverage Learning Experience Evaluation Score: %g' % (m.objVal/(3*4)))
            elif credit == 50:
                print('\nAverage Learning Experience Evaluation Score: %g' % (m.objVal/(3*5)))
        elif experience == "Coursework":
            if credit == 33:
                print('\nAverage Coursework Evaluation Score: %g' % (m.objVal/(2*3)))
            elif credit == 39 or credit == 44:
                print('\nAverage Coursework Evaluation Score: %g' % (m.objVal/(2*4)))
            elif credit == 50:
                print('\nAverage Coursework Evaluation Score: %g' % (m.objVal/(2*5)))
        elif experience == "Class Participation":
            if credit == 33:
                print('\nAverage Class Participation Evaluation Score: %g' % (m.objVal/(1*3)))
            elif credit == 39 or credit == 44:
                print('\nAverage Class Participation Evaluation Score: %g' % (m.objVal/(1*4)))
            elif credit == 50:
                print('\nAverage Class Participation Evaluation Score: %g' % (m.objVal/(1*5)))
        elif experience == "Prior Knowledge":
            if credit == 33:
                print('\nAverage Prior Knowledge Evaluation Score: %g' % (-m.objVal/(1*3)))
            elif credit == 39 or credit == 44:
                print('\nAverage Prior Knowledge Evaluation Score: %g' % (-m.objVal/(1*4)))
            elif credit == 50:
                print('\nAverage Prior Knowledge Evaluation Score: %g' % (-m.objVal/(1*5)))
    else:
        print('No solution:', m.status)

m.setParam('OutputFlag', 0)
m.optimize()
printSolution()


Average Overall Evaluation Score: 5.2245


In [28]:
(np.array(course)[[i for i, j in enumerate(m.getAttr("x", electives.values())) if j == 1]]).tolist()

['Behavioural Economics and Decision Making ',
 'Customer and Marketing Analytics ',
 'Innovation for Impact ',
 'Negotiation & Bargaining M']

# Optimize and Results

Here we run a loop to get results from every single possible preference to get a dataset for our dashboard building.

In [29]:
def get_electives(gifts, extra, subject, experience):
    # Build model
    m = gp.Model("electives")
    
    # Decision variables
    electives = m.addVars(course, vtype = GRB.BINARY, name = "electives")
    
    # Must constraints
    multi_stream_course = list(dict(filter(lambda elem: elem[1] > 1, 
                                           dict(Counter(data["Name"].tolist())).items())).keys())
    for name in multi_stream_course:
        index = data[data["Name"] == name].index
        m.addConstr(quicksum(electives[s] for s in index) <= 1)
        
    block_week_clashes = list(dict(filter(lambda elem: elem[1] > 1, 
                                          dict(Counter(data["Block Week Number"].tolist())).items())).keys())
    for week_number in block_week_clashes:
        index = data[data["Block Week Number"] == week_number].index
        m.addConstr(quicksum(electives[b] for b in index) <= 1)
        
    for term in data["Term"].unique().tolist():
        index = data[data["Term"] == term].index
        m.addConstrs(quicksum(electives[i]*data.loc[i,j] for i in index) <= 1 for j in class_schedules)
    
    for term in data["Term"].unique().tolist():
        index = data[data["Term"] == term].index
        m.addConstr(quicksum(electives[i] for i in index) <= 3)
    
    # Preference constraints
    index = data[data["Credit"] == 6].index
    if gifts == "Yes" and extra == "Yes":
        # Credit Constraints
        credit = 39
        m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
        # 6 Credit Course Constraint
        m.addConstr(quicksum(electives[i] for i in index) == 1)

    elif gifts == "Yes" and extra == "No":
        # Credit Constraints
        credit = 33
        m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
        # 6 Credit Course Constraint
        m.addConstr(quicksum(electives[i] for i in index) == 0)

    elif gifts == "No" and extra == "Yes":
        # Credit Constraints
        credit = 50
        m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
        # 6 Credit Course Constraint
        m.addConstr(quicksum(electives[i] for i in index) == 1)

    elif gifts == "No" and extra == "No":
        # Credit Constraints
        credit = 44
        m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in course) == credit)
        # 6 Credit Course Constraint
        m.addConstr(quicksum(electives[i] for i in index) == 0)
        
    if subject != "All":
        index = data[data["Subject"] == subject].index
        m.addConstr(quicksum(electives[i]*data.loc[i,"Credit"] for i in index) >= credit/2)
    
    if experience == "Overall":
        weight_dict = {"Q1":1, "Q2":1, "Q3":1, "Q4":1, "Q5":1, "Q6":1, "Q7":1}
    elif experience == "Learning Experience":
        weight_dict = {"Q1":1, "Q2":1, "Q3":1, "Q4":0, "Q5":0, "Q6":0, "Q7":0}
    elif experience == "Coursework":
        weight_dict = {"Q1":0, "Q2":0, "Q3":0, "Q4":1, "Q5":1, "Q6":0, "Q7":0}
    elif experience == "Class Participation":
        weight_dict = {"Q1":0, "Q2":0, "Q3":0, "Q4":0, "Q5":0, "Q6":1, "Q7":0}
    elif experience == "Prior Knowledge":
        weight_dict = {"Q1":0, "Q2":0, "Q3":0, "Q4":0, "Q5":0, "Q6":0, "Q7":1}
    
    # Objective function
    m.setObjective(quicksum(data.loc[j,i]*electives[j]*weight_dict[i]/7 for i in questions for j in course)-
                   quicksum(data.loc[j,"Q7"]*electives[j]*weight_dict["Q7"]/7 for j in course),
                   GRB.MAXIMIZE)
    m.setParam('OutputFlag', 0)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        return (np.array(course)[[i for i, j in enumerate(m.getAttr("x", electives.values())) if j == 1]]).tolist()
    else:
        return []

In [30]:
df = pd.DataFrame(columns=['GIFTs','Extra','Subject','Experience',
                           'Elective1','Elective2','Elective3','Elective4','Elective5'])

for a in ["Yes", "No"]:
    for b in ["Yes", "No"]:
        for c in ['Strategy & Entrepreneurship', 'Marketing', 'Finance', 'Management Science & Operations', 
                  'Organisational Behaviour', 'Accounting', 'Economics']:
            for d in ['Overall', 'Learning Experience', 'Coursework', 'Class Participation', 'Prior Knowledge']:
                temp = get_electives(a,b,c,d)
                
                try:
                    Elective1 = temp[0]
                except:
                    Elective1 = np.NaN
                    
                try:
                    Elective2 = temp[1]
                except:
                    Elective2 = np.NaN
                    
                try:
                    Elective3 = temp[2]
                except:
                    Elective3 = np.NaN
                
                try:
                    Elective4 = temp[3]
                except:
                    Elective4 = np.NaN
                    
                try:
                    Elective5 = temp[4]
                except:
                    Elective5 = np.NaN
                
                new_row = {'GIFTs':a, 'Extra':b, 'Subject':c, 'Experience':d, 
                           'Elective1':Elective1, 'Elective2':Elective2, 'Elective3':Elective3,
                           'Elective4':Elective4, 'Elective5':Elective5}
                df = df.append(new_row, ignore_index=True)
                

In [31]:
df.to_csv("Elective_Selection.csv",index=False)
df.head()

,GIFTs,Extra,Subject,Experience,Elective1,Elective2,Elective3,Elective4,Elective5
0,Yes,Yes,Strategy & Entrepreneurship,Overall,Advanced Competitive Strategy A,Customer and Marketing Analytics,Negotiation & Bargaining M,Strategic Innovation B,NaN
1,Yes,Yes,Strategy & Entrepreneurship,Learning Experience,Advanced Competitive Strategy A,Customer and Marketing Analytics,Strategic Innovation B,Value Investing B,NaN
2,Yes,Yes,Strategy & Entrepreneurship,Coursework,Innovation for Impact,Luxury Strategy,Measuring Impact in the Digital Economy,Strategic Innovation B,NaN
3,Yes,Yes,Strategy & Entrepreneurship,Class Participation,Entrepreneurship in Emerging Markets,Measuring Impact in the Digital Economy,Pathways to Start-up Success B,Value Investing B,NaN
4,Yes,Yes,Strategy & Entrepreneurship,Prior Knowledge,Advanced Competitive Strategy A,Customer Analytics Tools for Marketing Decisions,Elements of Machine Learning for Business,Financing the Entrepreneurial Business,NaN
